Podklady pro články Anety Martínkové na iROZHLAS.cz v prosinci 2023.

In [1]:
import os
import json
import pandas as pd

In [2]:
from src.tabulka import tabulka
from src.graf import graf

## Poslední týden v žebříčku bez…

In [8]:
vsechnytydny = [x for x in df['week'].drop_duplicates().to_list() if len(str(x)) == 7]

In [3]:
def kdynaposledne(x):
    pracovni = df[df['interpret_hlavni'] == x].groupby('week')['pozice'].min()
    pracovni = pracovni.reindex(vsechnytydny, fill_value=101).sort_index()
    return pracovni[pracovni > 100].tail(1).index[0]

In [5]:
df = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))
wikid = pd.read_json(os.path.join("wikidata","wikidata.json"))
df = df.merge(wikid, on="interpret_hlavni", how="outer")
df = df.sort_values(by='week')

In [6]:
dominatori = df.groupby('interpret_hlavni').size().nlargest(100).index.to_list()

In [9]:
doms = {}
for x in dominatori:
    doms[x] = kdynaposledne(x)

In [20]:
pd.DataFrame(pd.Series(doms).sort_values().head(5)).reset_index().rename(columns = {0 : 'Poslední týden mimo top 100'})

,index,Poslední týden mimo top 100
0,Chinaski,2016-52
1,Mirai,2017-04
2,Kryštof,2017-21
3,Miley Cyrus,2020-39
4,Jelen,2020-43


In [21]:
tabulka(pd.DataFrame(pd.Series(doms).sort_values().head(5)).reset_index().rename(columns = {"index": "Interpreti", 0 : 'Poslední týden mimo top 100'}), titulek="Nejdelší aktuální hitparádové šňůry")

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>Nejdelší aktuální hitparádové šňůry</caption><thead><tr style="text-align: right;"><th class="text-nowrap">Interpreti</th><th class="text-nowrap">Poslední týden mimo top 100</th></tr></thead><tbody class="text-sm"><tr><td>Chinaski</td><td>2016-52</td></tr><tr><td>Mirai</td><td>2017-04</td></tr><tr><td>Kryštof</td><td>2017-21</td></tr><tr><td>Miley Cyrus</td><td>2020-39</td></tr><tr><td>Jelen</td><td>2020-43</td></tr></tbody></table>'